In [1]:
from syft import nn
from syft import PhiTensor
from syft import GammaTensor
from syft import DataSubjectList
import numpy as np
from jax import numpy as jnp

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type Activation, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type Linear, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type BatchNorm, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySy

In [3]:
# stdlib
from typing import Dict
from typing import Tuple
from typing import Union

# third party
import numpy as np

# relative
activations = nn.activations
Layer = nn.layers.base.Layer

# from .. import activations
# from ....common.serde.serializable import serializable
# from ...autodp.phi_tensor import GammaTensor
# from ...autodp.phi_tensor import PhiTensor
# from .base import Layer


# @serializable(recursive_serde=True)
class BatchNorm(Layer):
    def __init__(self, epsilon: float=1e-6, momentum: float=0.9, axis: int=0, activation: activations.Activation=None):
        self.epsilon = epsilon
        self.momentum = momentum
        self.axis = axis
        self.activation = activations.get(activation)

        self.beta, self.dbeta = None, None
        self.gamma, self.dgamma = None, None
        self.cache = None
        self.input_shape = None
        self.out_shape = None


    def connect_to(self, prev_layer):
        n_in = prev_layer.out_shape[-1]
        self.beta = np.zeros((n_in,))
        self.gamma = np.ones((n_in,))
        self.out_shape = prev_layer.out_shape

    def forward(self, input: Union[PhiTensor, GammaTensor], *args, **kwargs):
        # N, D = x.shape

        # step1: calculate the mean
        xmu = input - input.mean(axis=0)
        # step2:
        # sq = xmu ** 2
        # var = 1. / N * np.sum(sq, axis=0)
        var = xmu.std(axis=0)

        # step3:
        sqrtvar = (var + self.epsilon).sqrt()
        ivar = 1.0 / sqrtvar

        # step4: normalization->x^
        xhat = xmu * ivar

        # step5: scale and shift
        gammax = xhat * self.gamma
        out = gammax + self.beta

        out = self.activation.forward(out) if self.activation is not None else out

        self.cache = (xhat, xmu, ivar, sqrtvar, var)

        return out

    def backward(self, pre_grad: Union[PhiTensor, GammaTensor], *args: Tuple, **kwargs: Dict):
        """
        If you get stuck, here's a resource:
        https://kratzert.github.io/2016/02/12/understanding-the-
        gradient-flow-through-the-batch-normalization-layer.html

        Note:
            - I removed the np.ones() at a few places where I
               thought it wasn't making a difference
            - I occasionally have kernel crashes on my 8GB machine
            when running this. Perhaps too many large temp vars?
            could also be due to too many large numbers.
        """

        xhat, xmu, ivar, sqrtvar, var = self.cache

        N, D, x, y = pre_grad.shape
        pre_grad = (pre_grad * self.activation.derivative()) if self.activation is not None else pre_grad

        # step6
        self.dbeta = pre_grad.sum(axis=0)
        dgammax = pre_grad
        self.dgamma = (dgammax * xhat).sum(axis=0)
        dxhat = dgammax * self.gamma

        # step5
        divar = (dxhat * xmu).sum(axis=0)
        dxmu1 = dxhat * ivar

        # step4
        dsqrtvar = -1.0 / (sqrtvar * sqrtvar) * divar
        inv_var_eps_sqrt = 1.0 / (var + self.epsilon).sqrt()
        dvar = dsqrtvar * 0.5 * inv_var_eps_sqrt

        # step3
        dsq = (pre_grad.ones_like()) * dvar * (1.0 / N)
        dxmu2 = xmu * dsq * 2

        # step2,
        dx1 = (dxmu1 + dxmu2)

        # step1,
        dmu = (dxmu1 + dxmu2).sum(axis=0) * -1
        dx2 = ((1/N) * pre_grad.ones_like()) * dmu

        # step0 done!
        dx = dx1 + dx2

        return dx

    @property
    def params(self):
        return self.beta, self.gamma

    @property
    def grades(self):
        return self.dbeta, self.dgamma


In [9]:
from syft import lazyrepeatarray as lra

def test_bnc():
    sh = (10, 3, 5, 5)

    dsl = DataSubjectList(
        one_hot_lookup=np.arange(10),
        data_subjects_indexed=np.random.choice(np.arange(10), size=sh)
    )
    smol_data = GammaTensor(
        child=jnp.array(np.random.rand(*sh) * 255),
        data_subjects=dsl,
        min_vals=lra(0, shape=sh),
        max_vals=lra(255, shape=sh))

    c = nn.Convolution(3, 3, input_shape=sh)
    c.connect_to()
    bn = nn.BatchNorm()
    bn.connect_to(c)
    print(c.out_shape)
    c_out = c.forward(smol_data)
    return bn.forward(c_out)


In [10]:
test_bnc()

(10, 3, 3, 3)
key (0, 0, 0, 0)
value [[[4 2 7]
  [7 9 4]
  [1 2 6]]

 [[5 8 3]
  [3 3 8]
  [5 9 4]]

 [[2 3 8]
  [5 4 7]
  [7 8 3]]]


ValueError: setting an array element with a sequence.